In [1]:
import os
import copy
import torch
import random
import pandas as pd

from itertools import product
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor

from torch.utils.tensorboard import SummaryWriter

def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
fix_random(42)

pca_t = False

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [4]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [5]:
# Applica la riduzione della dimensionalità con PCA
if pca_t == True:
    pca = PCA(n_components=0.95)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_val = pca.transform(X_val)
    X_test = pca.transform(X_test)
else:
    print ("PCA is not applied")

PCA is not applied


In [6]:
batch_sizes = [256]
n_epochs = [200]
# Dimension of the prediction  layer
n_d = [8, 16, 32]
#Dimension of the attention  layer
n_a = [8, 16, 32]
#Number of successive steps in the network
n_steps = [3, 5, 7]
# Number of independent GLU layer in each GLU block
n_indipendent = [2, 3]

params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb = len(batch_sizes)*len(n_d)*len(n_a) *len(n_steps) *len(n_indipendent)
print("Number of combinations: ", comb)

Number of combinations:  54


In [7]:
def get_model(n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_indipendent
    )
    return model

In [8]:
best_mse = float('inf')
best_model = None
best_params = None
iter = 0
for b, n_e, n_d, n_a, n_s, n_i in params:
    iter += 1
    print(f'\nIteration {iter}/{comb}')
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} - n_indipendent: {n_i}")
    
    model = get_model(b, n_e, n_d, n_a, n_s, n_i)
    #save results for each iteration with tensorboard
    log = f'bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}'

    if pca_t == True:
        if os.path.exists("./results/TabNet/pca/"+log):
            print("Model already trained. Skipping...")
            continue
        writer = SummaryWriter(f'results/TabNet/pca/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')
    else:
        if os.path.exists("./results/TabNet/no_pca/"+log):
            print("Model already trained. Skipping...")
            continue
        writer = SummaryWriter(f'results/TabNet/no_pca/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')

    #fit model
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        patience=10,
        batch_size=b,
        virtual_batch_size=128
    )

    # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
   #save hparams for each iteration with tensorboard
    writer.add_hparams(
        {'batch_size': b, 'n_epochs': n_e, 'n_d': n_d, 'n_a': n_a, 'n_steps': n_s, 'n_indipendent': n_i},
        {'hparam/mse': mse, 'hparam/r2': r2}
    )
    print('MSE:', mse)
    print('R2 Score:', r2)
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_params = (b, n_e, n_d, n_a, n_s, n_i)
        print('Best model updated')
    writer.close()


Iteration 1/54
Configuration batch size: 256 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 2


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.4039  | mse_mse: 0.69878 |  0:00:02s
epoch 1  | loss: 0.17248 | mse_mse: 0.25698 |  0:00:05s
epoch 2  | loss: 0.09203 | mse_mse: 0.15026 |  0:00:08s
epoch 3  | loss: 0.08277 | mse_mse: 0.16298 |  0:00:11s
epoch 4  | loss: 0.07352 | mse_mse: 0.12721 |  0:00:14s
epoch 5  | loss: 0.06916 | mse_mse: 0.10039 |  0:00:17s
epoch 6  | loss: 0.06224 | mse_mse: 0.10151 |  0:00:20s
epoch 7  | loss: 0.06907 | mse_mse: 0.09728 |  0:00:23s
epoch 8  | loss: 0.05664 | mse_mse: 0.06888 |  0:00:27s
epoch 9  | loss: 0.05378 | mse_mse: 0.06184 |  0:00:30s
epoch 10 | loss: 0.04512 | mse_mse: 0.05265 |  0:00:33s
epoch 11 | loss: 0.04459 | mse_mse: 0.05764 |  0:00:36s
epoch 12 | loss: 0.04484 | mse_mse: 0.03851 |  0:00:39s
epoch 13 | loss: 0.03895 | mse_mse: 0.04361 |  0:00:42s
epoch 14 | loss: 0.04081 | mse_mse: 0.03167 |  0:00:45s
epoch 15 | loss: 0.0333  | mse_mse: 0.03246 |  0:00:47s
epoch 16 | loss: 0.03019 | mse_mse: 0.02912 |  0:00:50s
epoch 17 | loss: 0.03134 | mse_mse: 0.02424 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.005526278937905899
R2 Score: 0.9750693538899957
Best model updated

Iteration 2/54
Configuration batch size: 256 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 3


...

MSE: 0.0072603892620705665
R2 Score: 0.967246279576661

Iteration 54/54
Configuration batch size: 256 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 - n_indipendent: 3


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 5.45549 | mse_mse: 0.71874 |  0:00:08s
epoch 1  | loss: 0.65631 | mse_mse: 0.41291 |  0:00:16s
epoch 2  | loss: 0.78668 | mse_mse: 0.16181 |  0:00:25s
epoch 3  | loss: 0.26863 | mse_mse: 0.10532 |  0:00:34s
epoch 4  | loss: 0.15552 | mse_mse: 0.19355 |  0:00:42s
epoch 5  | loss: 0.10522 | mse_mse: 0.15253 |  0:00:51s
epoch 6  | loss: 0.08631 | mse_mse: 0.10203 |  0:01:01s
epoch 7  | loss: 0.06631 | mse_mse: 0.1097  |  0:01:11s
epoch 8  | loss: 0.07118 | mse_mse: 0.07913 |  0:01:20s
epoch 9  | loss: 0.06991 | mse_mse: 0.06485 |  0:01:29s
epoch 10 | loss: 0.05877 | mse_mse: 0.06092 |  0:01:39s
epoch 11 | loss: 0.05573 | mse_mse: 0.06056 |  0:01:48s
epoch 12 | loss: 0.04904 | mse_mse: 0.04847 |  0:01:56s
epoch 13 | loss: 0.05202 | mse_mse: 0.04104 |  0:02:06s
epoch 14 | loss: 0.05476 | mse_mse: 0.03847 |  0:02:17s
epoch 15 | loss: 0.04874 | mse_mse: 0.03785 |  0:02:26s
epoch 16 | loss: 0.04528 | mse_mse: 0.03895 |  0:02:36s
epoch 17 | loss: 0.04502 | mse_mse: 0.03779 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.02091376308632036
R2 Score: 0.905651952753022


In [9]:
#pint the best model
print(best_model)
print("Best MSE: ", best_mse)
print("R2 score", r2_score(y_test, best_model.predict(X_test)))


TabNetRegressor(n_d=16, n_a=8, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=1128, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
Best MSE:  0.005357398783822059
R2 score 0.9758312212158716


In [10]:
#save the best model in a file csv
best_model.save_model('best_model_tabnet_nopca_256.csv')

Successfully saved model at best_model_tabnet_nopca_256.csv.zip


'best_model_tabnet_nopca_256.csv.zip'